# Using hcubature on uniform mesh of rectangular elements

## Import Packages

In [50]:
using ForwardDiff
using QuadGK
using HCubature 
using StaticArrays 
using LinearAlgebra
using BenchmarkTools
using Plots

In [4]:
# a point in 3D is a tuple of 3 coordinates 
# we here introduce static vectors that appear to be vital to reduce the number of allocations 
const Point3D = SVector{3,Float64};

## Section 1: Define Vector Potential Density  

In [10]:
# define vector potential density 
function vp_density(r, rprime, M)
    num = cross(M, (r - rprime))
    denom = norm(r - rprime)^3
    result = num/denom
    return result   
end

vp_density (generic function with 1 method)

In [13]:
M = [0.,0.,1.]
r = [1.,0.,0.]
rp = [0.,1.,0.]
vp_density(r,rp,M)

3-element Vector{Float64}:
  0.3535533905932737
  0.3535533905932737
 -0.0

In [16]:
M = [0.,0.,1.]
r = [1.,0.,0.]
t = 0.5 
hcubature(rp -> vp_density(r,rp,M), (0.0, 0.0, -t), (1.0, 1.0, t))[1]

3-element Vector{Float64}:
 1.551694097302116
 1.5516940972927296
 0.0

## Section 2: Perform Integration over Mesh 

In [17]:
# loop over midpoints of elements 
N = 5; h = 1/N;
xmesh = Vector(0:h:1)
ymesh = Vector(0:h:1)
for xi in xmesh[1:end-1] 
    for yj in ymesh[1:end-1] 
      mid = Point3D(xi+h/2,yj+h/2,0)
      # display(mid)
   end 
end 

In [52]:
# test integration assuming integrand for which answer can be checked analytically 
function settoone(rp)
    return 1.0
end 

N = 5; h = 1/N;
xmesh = Vector(0:h:1)
ymesh = Vector(0:h:1)
result = zeros(N,N)
for (i,xi) in enumerate(xmesh[1:end-1]) 
    for (j,yj) in enumerate(ymesh[1:end-1])  
      mid = Point3D(xi+h/2,yj+h/2,0)
      result[i,j] = hcubature(rp -> settoone(rp), (xi, yj, -t), (xi+h, yj+h, t))[1]
    end 
end

In [49]:
result

5×5 Matrix{Float64}:
 0.04  0.04  0.04  0.04  0.04
 0.04  0.04  0.04  0.04  0.04
 0.04  0.04  0.04  0.04  0.04
 0.04  0.04  0.04  0.04  0.04
 0.04  0.04  0.04  0.04  0.04

In [53]:
# assume r to coincide with midpoint of the element
N = 100; h = 1/N;
xmesh = Vector(0:h:1)
ymesh = Vector(0:h:1)
result = fill(zeros(3),N,N)
for (i,xi) in enumerate(xmesh[1:end-1]) 
    for (j,yj) in enumerate(ymesh[1:end-1])  
      mid = Point3D(xi+h/2,yj+h/2,0)
      result[i,j] = hcubature(rp -> vp_density(mid,rp,M), (xi, yj, -t), (xi+h, yj+h, t))[1]
    end 
end

In [41]:
result

5×5 Matrix{Vector{Float64}}:
 [-8.05737e-17, 1.0516e-12, 0.0]   …  [4.33192e-18, 1.06581e-12, 0.0]
 [-7.91297e-17, 5.68434e-14, 0.0]     [-1.44397e-18, 7.10543e-14, 0.0]
 [-8.34616e-17, -9.9476e-14, 0.0]     [-2.88794e-18, -1.13687e-13, 0.0]
 [-4.33192e-18, -9.9476e-14, 0.0]     [0.0, -1.13687e-13, 0.0]
 [5.77589e-18, 0.0, 0.0]              [-4.33192e-18, 0.0, 0.0]

In [56]:
# assume r to coincide with midpoint of the element
N = 5; h = 1/N;
xmesh = Vector(0:h:1)
ymesh = Vector(0:h:1)
result = fill(zeros(3),N,N)
for (i,xi) in enumerate(xmesh[1:end-1]) 
    for (j,yj) in enumerate(ymesh[1:end-1])  
      r = Point3D(xi,yj,-t)
      result[i,j] = hcubature(rp -> vp_density(r,rp,M), (xi, yj, -t), (xi+h, yj+h, t))[1]
    end 
end